In [1]:
# First, we have to setup Lynxius API key
import os
import sys
from getpass import getpass
sys.path.append("../")

if not (lynxius_api_key := os.getenv("LYNXIUS_API_KEY")):
    lynxius_api_key = getpass("🔑 Enter your Lynxius API key: ")

os.environ["LYNXIUS_API_KEY"] = lynxius_api_key
os.environ["LYNXIUS_BASE_URL"] = "https://platform.lynxius.ai"

🔑 Enter your Lynxius API key:  ········


In [2]:
# Makes it easier to iterate
%load_ext autoreload
%autoreload 2

In [3]:
from lynxius.client import LynxiusClient

client = LynxiusClient()

# Download the dataset (you can find the ID of your dataset on Lynxius online platform)
dataset_details = client.get_dataset_details(dataset_id="6e83cec5-d8d3-4237-af9e-8d4b7c71a2ce")

In [4]:
# Our sample LLM application
from datasets_utils import chatdoctor_v1

# Importing the evaluators
from lynxius.evals.bert_score import BertScore
from lynxius.evals.answer_correctness import AnswerCorrectness
from lynxius.evals.semantic_similarity import SemanticSimilarity
from lynxius.evals.custom_eval import CustomEval
from lynxius.evals.context_precision import ContextPrecision
from lynxius.evals.json_diff import JsonDiff

# ContextChunk represents a document retrieved from you RAG system
from lynxius.rag.types import ContextChunk

In [5]:
# Here we define sample RAG contexts.
# Retrieval of context documents depends on the RAG database that you're using
context = [
    ContextChunk(document="Avoid close contact with people who are sick. When you are sick, keep your distance from others to protect them from getting sick, too.", relevance=0.75),
    ContextChunk(document="If possible, stay home from work, school, and errands when you’re sick. You can go back to your normal activities when, for at least 24 hours, both are true:", relevance=0.31)
]

# Defaine tags to make it easier to locate these eval runs on the Lynxius platform
tags = ["notebook", "experiment"]

In [6]:
# Lynxius allows you to use your own evaluator templates
# Let's define and use one!

# When using a cusotm template, the only thing that you need to ensure is that
# the final verdict is printed at the very bottom of the resonse, with no other characters.
custom_eval_template = """
You are given a question, a reference answer and a candidate answer concerning a clinical matter.
You must determine if the candidate answer covers exactly the same content as the reference answer.
If the candidate answer contains additional information, or fails to mention something that is present
in the reference answer, your verdict should be 'incorrect'. Otherwise, your verdict should be 'correct'.
Provide a short explanation about how you arrived to your verdict. The verdict must be printed at the
very bottom of your response, on a new line, and it must not contain any extra characters.
Here is the data:
***********
Query: {query}
***********
Reference answer: {reference}
***********
Candidate answer: {output}
"""

In [7]:
# Define and run the evals
bert_score = BertScore("PR #111", level="word", presence_threshold=0.65, tags=tags)
answer_correctness = AnswerCorrectness("PR #111", tags=tags)
semantic_similarity = SemanticSimilarity("PR #111", tags=tags)
custom_eval = CustomEval("PR #111", name="clinical_correctness", prompt_template=custom_eval_template, tags=tags)
context_precision = ContextPrecision("PR #111", tags=tags)

for entry in dataset_details.entries:
    # Query our LLM
    actual_output = chatdoctor_v1(entry.query)

    # Add traces to the evals
    bert_score.add_trace(reference=entry.reference, output=actual_output, context=context)
    answer_correctness.add_trace(query=entry.query, reference=entry.reference, output=actual_output, context=context)
    semantic_similarity.add_trace(reference=entry.reference, output=actual_output, context=context)
    custom_eval.add_trace(values={"query": entry.query, "reference": entry.reference, "output": actual_output}, context=context)
    context_precision.add_trace(query=entry.query, reference=entry.reference, context=context)

# Run!
client.evaluate(bert_score)
client.evaluate(answer_correctness)
client.evaluate(semantic_similarity)
client.evaluate(custom_eval)
client.evaluate(context_precision)

'd76ff703-d58a-46b8-8732-434bc410326c'

In [8]:
# Define and run the JsonDiff eval
json_diff = JsonDiff("PR #111", tags=tags)

ref = {
    "prop1": True,
    "prop2": 0.85,
    "prop3": [
      {
        "prop4": 0.92,
        "prop5": 0.71
      },
      {
        "prop4": 0.22,
        "prop5": 1.0
      }
    ]
}
output = {
    "prop1": False,
    "prop2": 0.71,
    "prop3": [
      {
        "prop4": 0.89,
        "prop5": 0.55
      },
      {
        "prop4": 0.34,
        "prop5": 0.97
      }
    ]
}
# Weights is an optional parameter. 
# If not provided, each field will have an equal contribution to the overall score of every nested object.
weights = {
    "prop1": 0.5,
    "prop2": 0.5,
    "prop3": 1.0, # Default weights is 1.0 but we can also set it explicitly
}

json_diff.add_trace(reference=ref, output=output, weights=weights, context=context)
client.evaluate(json_diff)

'bcf65504-45d1-4c9e-a77b-2da77a40e698'

# Local evaluators
Below we demonstrate how to use the same evaluators but run them locally using your own OpenAI API key.

In [9]:
# We'll be using OpenAI locally so we have to set the API key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key:  ········


In [10]:
from lynxius.evals.local.bert_score import BertScoreLocal
from lynxius.evals.local.answer_correctness import AnswerCorrectnessLocal
from lynxius.evals.local.context_precision import ContextPrecisionLocal
from lynxius.evals.local.semantic_similarity import SemanticSimilarityLocal
from lynxius.evals.local.custom_eval import CustomEvalLocal

local_tags = ["local", "notebook", "experiment"]

bert_score_local = BertScoreLocal("PR #222", level="word", presence_threshold=0.65, tags=local_tags)
answer_correctness_local = AnswerCorrectnessLocal("PR #222", tags=local_tags)
context_precision_local = ContextPrecisionLocal("PR #222", tags=local_tags)
semantic_similarity_local = SemanticSimilarityLocal("PR #222", tags=local_tags)
custom_eval_local = CustomEvalLocal("PR #222", name="clinical_correctness", prompt_template=custom_eval_template, tags=local_tags)

for entry in dataset_details.entries:
    # Query our LLM
    actual_output = chatdoctor_v1(entry.query)

    # Add traces to the evals
    bert_score_local.add_trace(reference=entry.reference, output=actual_output, context=context)
    answer_correctness_local.add_trace(query=entry.query, reference=entry.reference, output=actual_output, context=context)
    semantic_similarity_local.add_trace(reference=entry.reference, output=actual_output, context=context)
    custom_eval_local.add_trace(values={"query": entry.query, "reference": entry.reference, "output": actual_output}, context=context)
    context_precision_local.add_trace(query=entry.query, reference=entry.reference, context=context)

# Evaluate locally!
bert_score_local.evaluate()
answer_correctness_local.evaluate()
semantic_similarity_local.evaluate()
custom_eval_local.evaluate()
context_precision_local.evaluate()

# Store the results in the Lynxius platform
client.store(bert_score_local)
client.store(answer_correctness_local)
client.store(semantic_similarity_local)
client.store(custom_eval_local)
client.store(context_precision_local)

'fb234332-c083-4234-ab77-73bf7acf693b'

In [11]:
from lynxius.evals.local.json_diff import JsonDiffLocal

json_diff_local = JsonDiffLocal("PR #222", tags=local_tags)
json_diff_local.add_trace(reference=ref, output=output, weights=weights, context=context)
json_diff_local.evaluate()
client.store(json_diff_local)

'bb369957-7f38-461b-9c2f-f65a4118e795'